# Fractopo – Fracture Network Analysis

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Cell contents only required for development env runs
from importlib.util import find_spec

if find_spec("fractopo") is None:
    import sys

    sys.path.append("..")

In [ ]:
from pathlib import Path
from shutil import rmtree, make_archive
from fractopo.analysis.network import Network
import fractopo.contour_grid as contour_grid
import matplotlib.pyplot as plt
import geopandas as gpd

plt.close()

## Data

Trace and target area data required. The paths can be urls to GeoJSON or local file paths to spatial filetypes (e.g. shapefile, geopackage). The name is used in plot labels and titles. 

1. Pass paths to your **validated** trace and area data here and name the analysis. E.g.,

``` {python}
trace_data = "traces.gpkg"
area_data = "target_area.gpkg"
name = "my-analysis-name"
```
   * The path is relative to the notebook directory. To make things easy you should've copied the notebook the working directory which       either directly contains your trace and area data or has the folder that does. Tab-completion works here aswell.
    
   * Note that the analysis name is used to create a folder like such `results/my-analysis-name` where all analysis results are saved to. If such a folder exists, all contents        will be overridden in the `my-analysis-name` folder.
   * Note that the path is inside quotes. These are mandatory.

In [ ]:
trace_data = ""
area_data = ""
name = ""

The defaults in the next cell are only applied if no parameters are given to the above cell. This will result in a **default** analysis of a trace and area data downloaded from the urls.

In [ ]:
if len(trace_data) == 0:
    # Set defaults
    # Trace and target area data available on GitHub
    trace_data = "https://raw.githubusercontent.com/nialov/fractopo/master/tests/sample_data/KB11/KB11_traces.geojson"
    area_data = "https://raw.githubusercontent.com/nialov/fractopo/master/tests/sample_data/KB11/KB11_area.geojson"
    # Name the dataset
    name = "KB11"

2. The preselected analysis set can now be run! To run the notebook, click on the double-right-arrow on at the top of the notebook below the tab bar and click Restart.

   * You can see the cells being executed with numbers appearing on the left.
   * Some cells will take much longer than others depending on code execution time.
   * Scroll down the notebook as the numbers appear until all cells have been reached.
   * If the analysis throws errors they will appear in big red boxes.
  
**However**, you might want to change some defaults such as azimuth set ranges and set names and contour grid cell width. Scroll down to headers with ``USER INPUT:`` prefixes and follow the instructions there to configure default values.

3. If no errors occur during running the results of the analysis will be in `results/my-analysis-name` folder (and an archived .zip).

   * The folder will contain plots and spatial data files:
   
       * Rose plot of trace azimuths, length-weighted
       * Length distribution plots
       * XYI-plots
       * Branches and nodes
       * Contour grids
       * Etc.
   
   * The folder has been also archived as a .zip file for easy downloading (`results/my-analysis-name.zip`).
   
   * If errors do occur:
       
       * Check the error message that occurred for possible solutions.
       * Check that the trace and area paths are correct.
       * You can restart the run from the same double-right-arrow symbol.
       * Report errors that you can't solve at https://github.com/nialov/fractopo-help/issues

4. Some analyses will be run with default settings which might not fit your dataset.

    * This is especially the case for contour grids (grid cell size).
    * Scroll down to the contour grid section to configure if the results are not to your liking.

In [ ]:
# Make/overwrite results dir
results_dir = Path("results") / name
if results_dir.exists():
    rmtree(results_dir)
results_dir.mkdir(parents=True)

In [ ]:
# Use geopandas to load data from urls/paths
traces = gpd.read_file(trace_data)
area = gpd.read_file(area_data)

In [ ]:
area.total_bounds

In [ ]:
def focus_plot_to_bounds(ax, total_bounds):
    """ Focus plot to given bounds. """
    xmin, ymin, xmax, ymax = total_bounds
    extend_x = (xmax - xmin) * 0.05
    extend_y = (ymax - ymin) * 0.05
    ax.set_xlim(xmin - extend_x, xmax + extend_x)
    ax.set_ylim(ymin - extend_y, ymax + extend_y)
    return ax


def save_fig(fig, results_dir: Path, name: str):
    """ Save figure as svg image to results dir. """
    fig.savefig(results_dir / f"{name}.svg", bbox_inches="tight")


def as_gpkg_and_shp(geodataframe, name):
    """ Save geodataframe as GeoPackage and as shapefile. """
    geodataframe.to_file(results_dir / f"{name}.gpkg", driver="GPKG")
    shp_dir = results_dir / f"{name}_as_shp"
    shp_dir.mkdir()
    geodataframe.to_file(shp_dir / f"{name}.shp")

## Visualizing trace map data

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
traces.plot(ax=ax, color="blue")
area.boundary.plot(ax=ax, color="red")
ax = focus_plot_to_bounds(ax, area.total_bounds)
save_fig(fig, results_dir, "base_visualization")

## Create Network

### USER INPUT: Pass your own azimuth sets

You may pass your own azimuth sets here for e.g., cross-cutting and abutting relationship analysis. 

You must pass two types of values:

1. Pass a range e.g., `(0, 60)` means the set contains lines with azimuths between 0 and 60 degrees.

    * The range can circle around zero e.g. a range `(170, 30)` is accepted

2. Pass the name for the range. Short names, possibly numerical are preferred e.g., `"1"` or `"A"`.
3. Follow the below shown format and do not remove parenthesis or quotes. The inputs must be valid Python code.

Each range must have an associated name. The inputted ranges must be in the same order as the names.

#### Examples:

Contains three sets:

``` {python}
azimuth_set_ranges = (
    (0, 60),
    (60, 120),
    (120, 180),
)
azimuth_set_names = (
    "1", 
    "2", 
    "3",
)
```

Contains two sets:

``` {python}
azimuth_set_ranges = (
    (0, 60),
    (170, 30),
)
azimuth_set_names = (
    "A", 
    "B",
)
```

In [ ]:
# These are the default values. Input your values here and change the defaults (if needed).
azimuth_set_ranges = (
    (0, 60),
    (60, 120),
    (120, 180),
)
azimuth_set_names = (
    "1",
    "2",
    "3",
)

This next cell automatically checks your azimuth set inputs for basic errors.

In [ ]:
assert len(azimuth_set_ranges) == len(azimuth_set_names)
for set_range in azimuth_set_ranges:

    assert len(set_range) == 2
    assert isinstance(set_range, tuple)

assert all([isinstance(val, str) for val in azimuth_set_names])

In [ ]:
# Create Network and automatically determine branches and nodes
network = Network(
    traces,
    area,
    name=name,
    determine_branches_nodes=True,
    snap_threshold=0.001,
    azimuth_set_ranges=azimuth_set_ranges,
    azimuth_set_names=azimuth_set_names,
    # If the target area is a circle, can be changed to True
    circular_target_area=False,
    truncate_traces=True,
)

## Visualizing branches and nodes

In [ ]:
from fractopo.general import CC_branch, CI_branch, II_branch, X_node, Y_node, I_node


# Function to determine color for each branch and node type
def assign_colors(feature_type: str):
    if feature_type in (CC_branch, X_node):
        return "green"
    if feature_type in (CI_branch, Y_node):
        return "blue"
    if feature_type in (II_branch, I_node):
        return "black"
    return "red"

| Branch or Node Type | Color |
|---------------------|-------|
| C - C, X            | Green |
| C - I, Y            | Blue  |
| I - I, I            | Black |
| Other               | Red   |

### Branches

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
network.branch_gdf.plot(
    colors=[assign_colors(bt) for bt in network.branch_types], ax=ax
)
area.boundary.plot(ax=ax, color="red")
save_fig(fig, results_dir, "branches_visualization")

### Nodes

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
# Traces
network.trace_gdf.plot(ax=ax, linewidth=0.5)
# Nodes
network.node_gdf.plot(
    c=[assign_colors(bt) for bt in network.node_types], ax=ax, markersize=10
)
area.boundary.plot(ax=ax, color="red")
ax = focus_plot_to_bounds(ax, area.total_bounds)
save_fig(fig, results_dir, "nodes_visualization")

## Rose plots

In [ ]:
# Plot azimuth rose plot of fracture traces
azimuth_bin_dict, fig, ax = network.plot_trace_azimuth()
save_fig(fig, results_dir, "trace_length_weighted_rose_plot")

In [ ]:
network.trace_azimuth_set_counts

In [ ]:
# Plot azimuth rose plot of fracture branches
azimuth_bin_dict, fig, ax = network.plot_branch_azimuth()
save_fig(fig, results_dir, "branch_length_weighted_rose_plot")

In [ ]:
network.branch_azimuth_set_counts

## Length distributions

### Trace length distribution

In [ ]:
# Fit for traces
fit_traces = network.trace_lengths_powerlaw_fit()

In [ ]:
# Plot length distribution fits (powerlaw, exponential and lognormal) of fracture traces
fit, fig, ax = network.plot_trace_lengths()
save_fig(fig, results_dir, "trace_length_distribution_fits")

In [ ]:
# Fit properties
print(f"Automatically determined powerlaw cut-off: {fit_traces.xmin}")
print(f"Powerlaw exponent: {fit_traces.alpha - 1}")
print(
    f"Proportion of data cut off by cut off: {network.trace_lengths_cut_off_proportion()}"
)
print(
    f"Compare powerlaw fit to lognormal: R, p = {fit_traces.distribution_compare('power_law', 'lognormal')}"
)

### Branch length distribution

In [ ]:
# Fit for branches
fit_branches = network.branch_lengths_powerlaw_fit()

In [ ]:
# Plot length distribution fits (powerlaw, exponential and lognormal) of fracture branches
fit, fig, ax = network.plot_branch_lengths()
save_fig(fig, results_dir, "branch_length_distribution_fits")

In [ ]:
# Fit properties
print(f"Automatically determined powerlaw cut-off: {fit_branches.xmin}")
print(f"Powerlaw exponent: {fit_branches.alpha - 1}")
print(
    f"Proportion of data cut off by cut off: {network.branch_lengths_cut_off_proportion()}"
)
print(
    f"Compare powerlaw fit to lognormal: R, p = {fit_branches.distribution_compare('power_law', 'lognormal')}"
)

## Crosscutting and abutting relationships

In [ ]:
# Sets are defaults
print(f"Azimuth set names: {network.azimuth_set_names}")
print(f"Azimuth set ranges: {network.azimuth_set_ranges}")

In [ ]:
# Plot crosscutting and abutting relationships between azimuth sets
figs, fig_axes = network.plot_azimuth_crosscut_abutting_relationships()
for i, fig in enumerate(figs):
    save_fig(fig, results_dir, f"azimuth_set_relationships_{i}")

## Node and branch proportions

In [ ]:
network.node_counts

In [ ]:
# Plot ternary XYI-node proportion plot
fig, ax, tax = network.plot_xyi()
save_fig(fig, results_dir, "xyi_ternary_plot")

In [ ]:
network.branch_counts

In [ ]:
# Plot ternary branch (C-C, C-I, I-I) proportion plot
fig, ax, tax = network.plot_branch()
save_fig(fig, results_dir, "branch_ternary_plot")

## Data to files

In [ ]:
# Save traces, branches and nodes.
as_gpkg_and_shp(network.trace_gdf, "traces")
as_gpkg_and_shp(network.branch_gdf, "branches")
as_gpkg_and_shp(network.node_gdf, "nodes")

# General topological and geometric parameters

In [ ]:
def get_crs(traces, area):
    for crs_maybe in (traces.crs, area.crs):
        if crs_maybe is not None:
            return crs_maybe
    return None


params_point = gpd.GeoDataFrame(
    [{**network.parameters, "geometry": area.geometry.values[0].representative_point()}]
)
crs_maybe = get_crs(traces, area)
if crs_maybe is not None:
    params_point.set_crs(crs_maybe)
as_gpkg_and_shp(params_point, "params")

# Contour grids for target area

## USER INPUT: Configuring of contour grids

Configuration of contour grids happens here.

* `cell_width` is the width of the sample cell in the grid. Change the number from the default (unit is meters in ETRS-TM35FIN) to your liking
* `snap_threshold` is the snapping threshold for your dataset. See the `tracevalidate` guide for more info (for ETRS-TM35FIN crs and with drone orthophotography data, values between 0.01 and 0.001 are fine. Results may vary).

In [ ]:
sampled_grid = contour_grid.run_grid_sampling(
    traces=network.trace_gdf,
    branches=network.branch_gdf,
    nodes=network.node_gdf,
    cell_width=2.0,
    snap_threshold=0.01,
)
if crs_maybe is not None:
    sampled_grid.set_crs(crs_maybe)
as_gpkg_and_shp(sampled_grid, "contour_grid")

In [ ]:
sampled_grid.columns

In [ ]:
# From https://geopandas.org/mapping.html
from mpl_toolkits.axes_grid1 import make_axes_locatable


def plot_contour(column: str):
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    sampled_grid.plot(
        column=column, legend=True, cax=cax, ax=ax, legend_kwds={"label": column}
    )
    return fig, ax

In [ ]:
fig, ax = plot_contour("Fracture Intensity P21")
save_fig(fig, results_dir, "P21_contour")

In [ ]:
fig, ax = plot_contour("Connections per Branch")
save_fig(fig, results_dir, "Connections_per_branch_contour")

In [ ]:
# Zip the folder in results.
base_zip_path = Path("results") / f"{name}"
full_zip_path = base_zip_path.with_suffix(".zip")
if full_zip_path.exists():
    full_zip_path.unlink()
make_archive(base_zip_path, "zip", results_dir)